In [1]:
import networkx as nx
from networkx.algorithms import community
from pyvis.network import Network
import math
import numpy as np

In [2]:
### Function to check condition 1: if node has atleast 1 edge connecting to the other community it is not part of
#   pre-condition( 1. graph has only 2 communities
#                   2. n is a node in the graph)  
###
def cond1(n, model_graph, communities):
    for b in (0,1):
        # loops though neighbours of node n
        for a in list(model_graph.adj[n]):
            # checks if neighbour is of a different community
            if n in communities[b] and not(a in communities[b]):
                return True
    return False

In [3]:
### Function to check condition 2: if node is connected to atleast one internal node of its community
#   pre-condition( 1. graph has only 2 communities
#                   2. n is a node in the graph)
###
def cond2(n, model_graph, communities):
    for k in (0,1):
        # loops over neighbours of node n
        for a in list(model_graph.adj[n]):
            m=0 # counts neighbours from a different community
            # checks if neighbour is part of same community
            if (a in communities[k] and n in communities[k]):
                # loops over neighbours of the neighbour node a
                for b in list(model_graph.adj[a]):
                    # checks if node a is an internal node
                    if not(b in communities[k]):
                        # increment since node a is conected to the other community(not an internal node)
                        m+=1
                if m==0:
                    # returns when first internal neighbour node found
                    return True
    return False

In [4]:
### Sorts boundary and internal nodes from each community and returns 2 lists.
#   pre-condition( graph has only 2 communities)
###
def node_sets(communities,graph):
    boundary1=[] # stores boundary nodes of community 1
    boundary2=[] # stores boundary nodes of community 2
    for n in nx.nodes(graph):
        # checks if both conditions for node to be boundary node is satisfied
        if cond1(n, graph, communities) and cond2(n,graph, communities):
            if n in communities[0]:                
                boundary1.append(n)
            else: # if n in communities[1]:
                boundary2.append(n)
    boundary=[boundary1,boundary2]
    internal=[list(communities[0]),list(communities[1])]
    # removes list of boundary node from internal node
    # internal nodes I_i formula: I_i = G_i - B_i
    for a in range(0,1):
        # loops over nodes in community
        for n in communities[a]:
            # checks if the particular node also appears in set of boundary nodes for the same community
            if n in boundary[a]:
                internal[a].remove(n)
    return boundary, internal

In [5]:
### Sorts boundary and internal edges from each community and returns 2 lists.
#   pre-condition( 1. graph has only 2 communities
#                   2. bound and internal are sets of nodes in the graph)
###
def edges_sets(bound, internal, graph):
   
    eb=[] # distinct edges between boundary nodes
    # edges of community 1
    for n in range(len(bound[0])):
        sub_b=[]
        # loop over neighbours of node n
        for neigh in list(graph.adj[bound[0][n]]):
            if neigh in bound[1]:
                # adds edges linked to neighbours from different community
                sub_b.append([bound[0][n],neigh])
        eb.append(sub_b)
    # edges of community 2
    for n in range(len(bound[1])):
        sub_b=[]
        for neigh in list(graph.adj[bound[1][n]]):
            if neigh in bound[0]:
                sub_b.append([bound[1][n],neigh])
        eb.append(sub_b)
    
    eint=[] # distinct edges between internal nodes
    # edges of community 1
    for n in range(len(bound[0])):
        sub_int=[]
        for neigh in list(graph.adj[bound[0][n]]):
            if (neigh in internal[0]) or (neigh in internal[1]):
                sub_int.append([bound[0][n],neigh])  
        eint.append(sub_int)
    # edges of community 2
    for n in range(len(bound[1])):
        sub_int=[]
        for neigh in list(graph.adj[bound[1][n]]):
            if (neigh in internal[0]) or (neigh in internal[1]):
                sub_int.append([bound[1][n],neigh])  
        eint.append(sub_int)
    return eb, eint

In [6]:
### Computes P value from formula. 
###
def p_value(boundary,e_bound,e_inter):
    mixed_bound=[]
    mixed_bound.extend(boundary[0])
    mixed_bound.extend(boundary[1])
    abs_b=len(mixed_bound)
    sum=0
    for n in range(len(mixed_bound)):
        sum=sum-0.5+len(e_inter[n])/(len(e_inter[n])+len(e_bound[n]))
    p_value=sum/abs_b
    return p_value

In [7]:
# create karate club graph
model_graph = nx.karate_club_graph()
# create the 2 communities based on modularity/resolution mentioned in the paper(0.35)?
communities = community.naive_greedy_modularity_communities(model_graph, resolution=0.3)
# sorts boundary nodes and internal nodes
[boundary,internal]=node_sets(communities,model_graph)
[e_bound,e_inter]=edges_sets(boundary,internal,model_graph)
P=p_value(boundary,e_bound,e_inter)
print(P)


TypeError: naive_greedy_modularity_communities() got an unexpected keyword argument 'resolution'